### imports 

In [ ]:

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import numpy
# Models

from transformers import AutoTokenizer

# Training

import tensorflow

# Evaluation

# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns

from transformers import TFAutoModelForSequenceClassification

from datasets import load_dataset


### configs

### preproccess the raw data into title text combinations

### preprocess data 

In [ ]:
raw_datasets = load_dataset('csv', data_files={ "train": 'data/train.csv', "test": 'data/test.csv', "valid": 'data/valid.csv'})

### tokenize data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["titletext"], padding="max_length", truncation=True)

In [ ]:

# raw_datasets = raw_datasets.map(lambda batch: { "binary_labels": tensorflow.keras.utils.to_categorical(batch["labels"],2) }, batched=True) 
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(250))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(250))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)


In [ ]:
tf_train_dataset = small_train_dataset.remove_columns(["titletext"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["titletext"]).with_format("tensorflow")

In [ ]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tensorflow.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["labels"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tensorflow.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["labels"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [ ]:
model.compile(
    optimizer="adam",
    loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tensorflow.keras.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=2)

https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b

https://www.tensorflow.org/text/tutorials/classify_text_with_bert

word embeddings + LSTM https://towardsdatascience.com/text-classification-on-disaster-tweets-with-lstm-and-word-embedding-df35f039c1db

https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data

https://huggingface.co/transformers/training.html

In [ ]:
# # Prepare columns
# df_raw['label'] = (df_raw['OpenStatus'] == 'open').astype('int')
# df_raw['titletext'] = df_raw['Title'] + ". " + df_raw['BodyMarkdown']
# # df_raw = df_raw.reindex(columns=['label', 'Title', 'BodyMarkdown', 'titletext'])

# # # Drop rows with empty text
# # df_raw.drop( df_raw[df_raw.text.str.len() < 5].index, inplace=True)

# # Trim text and titletext to first_n_words
# df_raw['BodyMarkdown'] = df_raw['BodyMarkdown'].apply(trim_string)
# df_raw['titletext'] = df_raw['titletext'].apply(trim_string) 

# # Split according to label
# df_open = df_raw[df_raw['label'] == 1]
# df_closed = df_raw[df_raw['label'] == 0]

# # Train-test split
# df_open_full_train, df_open_test = train_test_split(df_open, train_size = train_test_ratio, random_state = 1)
# df_closed_full_train, df_closed_test = train_test_split(df_closed, train_size = train_test_ratio, random_state = 1)

# # Train-valid split
# df_open_train, df_open_valid = train_test_split(df_open_full_train, train_size = train_valid_ratio, random_state = 1)
# df_closed_train, df_closed_valid = train_test_split(df_closed_full_train, train_size = train_valid_ratio, random_state = 1)

# # Concatenate splits of different labels
# df_train = pd.concat([df_open_train, df_closed_train], ignore_index=True, sort=False)
# df_valid = pd.concat([df_open_valid, df_closed_valid], ignore_index=True, sort=False)
# df_test = pd.concat([df_open_test, df_closed_test], ignore_index=True, sort=False)

# # Write preprocessed data
# df_train.to_csv("./data" + '/train.csv', index=False)
# df_valid.to_csv(destination_folder + '/valid.csv', index=False)
# df_test.to_csv(destination_folder + '/test.csv', index=False)

